In [1]:
import numpy as np
import pandas as pd
pokemon = pd.read_csv(r"C:\Users\Krisna Sidiq\Downloads\Recruitment\Singa.id\pokemon.csv")
pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [2]:
pokemon = pokemon.rename(columns={"#": "ID"})
pokemon.head()

,ID,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [3]:
combat = pd.read_csv(r"C:\Users\Krisna Sidiq\Downloads\Recruitment\Singa.id\combats.csv")
combat.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [4]:
print(pokemon.shape)
print(combat.shape)

(800, 12)
(50000, 3)


Cleaning Data

In [5]:
pokemon.isnull().sum()

ID              0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [6]:
print(pokemon[pokemon['Name'].isnull()])

    ID Name    Type 1 Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
62  63  NaN  Fighting    NaN  65     105       60       60       70     95   

    Generation  Legendary  
62           1      False  


In [7]:
# check before 62
pokemon.at[61,'Name']

'Mankey'

In [8]:
# Fill in the name of pokemon according to what is on the internet
pokemon.at[62,'Name']= "Primeape"
print(pokemon.iloc[62])

ID                  63
Name          Primeape
Type 1        Fighting
Type 2             NaN
HP                  65
Attack             105
Defense             60
Sp. Atk             60
Sp. Def             70
Speed               95
Generation           1
Legendary        False
Name: 62, dtype: object


In [9]:
pokemon.duplicated(keep='first')

0      False
1      False
2      False
3      False
4      False
       ...  
795    False
796    False
797    False
798    False
799    False
Length: 800, dtype: bool

In [10]:
combat.isnull().sum()

First_pokemon     0
Second_pokemon    0
Winner            0
dtype: int64

In [11]:
combat.duplicated(keep='first')

0        False
1        False
2        False
3        False
4        False
         ...  
49995    False
49996    False
49997     True
49998    False
49999    False
Length: 50000, dtype: bool

Organizing Data

In [12]:
winQty = combat.groupby('Winner').count()
winQty

,First_pokemon,Second_pokemon
Winner,,
1,37,37
2,46,46
3,89,89
4,70,70
5,55,55
...,...,...
796,39,39
797,116,116
798,60,60


In [13]:
winQty =winQty.drop('Second_pokemon',axis=1)
winQty =winQty.rename(columns={"First_pokemon": "Win_Qty"})
winQty

,Win_Qty
Winner,
1,37
2,46
3,89
4,70
5,55
...,...
796,39
797,116
798,60


In [14]:
countFirst = combat.groupby('Second_pokemon').count()
countSecond = combat.groupby('First_pokemon').count()
countFirst

,First_pokemon,Winner
Second_pokemon,,
1,63,63
2,66,66
3,64,64
4,63,63
5,62,62
...,...,...
796,56,56
797,67,67
798,59,59


In [15]:
winQty['Total_Fights'] = countFirst.Winner + countSecond.Winner
winQty['Win_Percentage']= winQty.Win_Qty/winQty['Total_Fights']
winQty

,Win_Qty,Total_Fights,Win_Percentage
Winner,,,
1,37,133,0.278195
2,46,121,0.380165
3,89,132,0.674242
4,70,125,0.560000
5,55,112,0.491071
...,...,...,...
796,39,105,0.371429
797,116,131,0.885496
798,60,119,0.504202


In [23]:
summary = pd.merge(pokemon, winQty, left_on='ID', right_index = True, how='left')
summary

,ID,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Win_Qty,Total_Fights,Win_Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,133.0,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,121.0,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,132.0,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,125.0,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,112.0,0.491071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,39.0,105.0,0.371429
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,116.0,131.0,0.885496
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True,60.0,119.0,0.504202
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,89.0,144.0,0.618056


Analysis

In [24]:
features = ['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Win_Percentage']
corr = summary[features]
corr

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Win_Percentage
0,45,49,49,65,65,45,0.278195
1,60,62,63,80,80,60,0.380165
2,80,82,83,100,100,80,0.674242
3,80,100,123,122,120,80,0.560000
4,39,52,43,60,50,65,0.491071
...,...,...,...,...,...,...,...
795,50,100,150,100,150,50,0.371429
796,50,160,110,160,110,110,0.885496
797,80,110,60,150,130,70,0.504202
798,80,160,60,170,130,80,0.618056


In [25]:
df_corr = corr.corr()
df_corr

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Win_Percentage
HP,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952,0.258006
Attack,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240,0.500181
Defense,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227,0.129426
Sp. Atk,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018,0.478940
Sp. Def,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133,0.324218
Speed,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000,0.937742
Win_Percentage,0.258006,0.500181,0.129426,0.478940,0.324218,0.937742,1.000000


Save Data

In [26]:
summary.to_csv('summary.csv', index=False)  

Preparing Data for Prediction

In [58]:
pokemon = summary
pokemon

,ID,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Win_Qty,Total_Fights,Win_Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,133.0,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,121.0,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,132.0,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,125.0,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,112.0,0.491071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,39.0,105.0,0.371429
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,116.0,131.0,0.885496
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True,60.0,119.0,0.504202
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,89.0,144.0,0.618056


In [59]:
pokemon.isna().sum()

ID                  0
Name                0
Type 1              0
Type 2            386
HP                  0
Attack              0
Defense             0
Sp. Atk             0
Sp. Def             0
Speed               0
Generation          0
Legendary           0
Win_Qty            17
Total_Fights       17
Win_Percentage     17
dtype: int64

In [60]:
# Delete Nan in last three columns
pokemon = pokemon.dropna(subset=['Win_Qty'])
pokemon

,ID,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Win_Qty,Total_Fights,Win_Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,133.0,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,121.0,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,132.0,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,125.0,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,112.0,0.491071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,39.0,105.0,0.371429
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,116.0,131.0,0.885496
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True,60.0,119.0,0.504202
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,89.0,144.0,0.618056


In [61]:
pokemon.isna().sum()

ID                  0
Name                0
Type 1              0
Type 2            378
HP                  0
Attack              0
Defense             0
Sp. Atk             0
Sp. Def             0
Speed               0
Generation          0
Legendary           0
Win_Qty             0
Total_Fights        0
Win_Percentage      0
dtype: int64

In [62]:
# X is features, y is win_percentage
X = pokemon.iloc[:, 4:11].values
y = pokemon.iloc[:, -1].values

In [63]:
X

array([[ 45,  49,  49, ...,  65,  45,   1],
       [ 60,  62,  63, ...,  80,  60,   1],
       [ 80,  82,  83, ..., 100,  80,   1],
       ...,
       [ 80, 110,  60, ..., 130,  70,   6],
       [ 80, 160,  60, ..., 130,  80,   6],
       [ 80, 110, 120, ...,  90,  70,   6]], dtype=int64)

In [64]:
y

array([0.27819549, 0.38016529, 0.67424242, 0.56      , 0.49107143,
       0.54237288, 0.86466165, 0.85611511, 0.84444444, 0.16239316,
       0.41843972, 0.57638889, 0.14782609, 0.09774436, 0.625     ,
       0.23423423, 0.13076923, 0.625     , 0.96638655, 0.37301587,
       0.52892562, 0.86153846, 0.94444444, 0.48148148, 0.72142857,
       0.44927536, 0.83739837, 0.36641221, 0.576     , 0.68382353,
       0.8677686 , 0.53333333, 0.16806723, 0.38513514, 0.54098361,
       0.25      , 0.3984375 , 0.56428571, 0.15748031, 0.39837398,
       0.41085271, 0.81102362, 0.11538462, 0.37931034, 0.7578125 ,
       0.06293706, 0.21705426, 0.35483871, 0.0942029 , 0.19480519,
       0.22377622, 0.65714286, 0.53968254, 0.84375   , 0.65      ,
       0.85833333, 0.33098592, 0.68695652, 0.46610169, 0.78703704,
       0.45      , 0.79824561, 0.72      , 0.52307692, 0.32407407,
       0.66923077, 0.77142857, 0.78181818, 0.14      , 0.25806452,
       0.43965517, 0.21296296, 0.37190083, 0.50318471, 0.80341

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape

(626, 7)

Model

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [67]:
model_linear = LinearRegression()
model_SVM = SVR(kernel = 'linear')
model_tree = DecisionTreeRegressor(random_state = 0)
model_forest = RandomForestRegressor(n_estimators = 10, random_state = 0)

In [68]:
def train(X_train, X_test, y_train, y_test,model):
    model.fit(X_train, y_train)
    print("Score:" , model.score(X_train, y_train))

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    print("MAE: " + str(mae))

In [69]:
print("Linear Regression")
train(X_train, X_test, y_train, y_test,model_linear)

Linear Regression
Score: 0.9086804453503223
MAE: 0.06130011878545476


In [70]:
print("SVM")
train(X_train, X_test, y_train, y_test,model_SVM)

SVM
Score: 0.9070061319249877
MAE: 0.06372492881825428


In [71]:
print("Decision Tree")
train(X_train, X_test, y_train, y_test,model_tree)

Decision Tree
Score: 0.9997364519875158
MAE: 0.056058422790240606


In [72]:
print("Random Forest")
train(X_train, X_test, y_train, y_test,model_forest)

Random Forest
Score: 0.9904321126678309
MAE: 0.04919730733189687


Based on the score and MAE, the random forest is the best model because it has a low MAE but a high score

Build Prediction

In [73]:
pokemon

,ID,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Win_Qty,Total_Fights,Win_Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,133.0,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,121.0,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,132.0,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,125.0,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,112.0,0.491071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,39.0,105.0,0.371429
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,116.0,131.0,0.885496
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True,60.0,119.0,0.504202
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,89.0,144.0,0.618056


In [114]:
def battle(pokemon1_name, pokemon2_name):
    if (pokemon1_name not in pokemon.values) or (pokemon2_name not in pokemon.values):
        print('Maaf, data tidak ditemukan.')
        return

    # get data for the two pokemon
    pokemon1_data = pokemon.loc[pokemon['Name']==pokemon1_name, 'HP':'Generation']
    pokemon2_data = pokemon.loc[pokemon['Name']==pokemon2_name, 'HP':'Generation']

    # combine the X data into a single array
    X = pd.concat([pd.DataFrame(pokemon1_data), pd.DataFrame(pokemon2_data)], axis=0, ignore_index=True)

    # make a prediction using the trained model
    prediction = model_forest.predict(X)
    
    # calculate the probability of each pokemon winning
    prob1 = sum(prediction[:len(pokemon1_data)]) / len(pokemon1_data)
    prob2 = sum(prediction[len(pokemon1_data):]) / len(pokemon2_data)
    
    # print the result
    if prob1 > prob2:
        print(f"{pokemon1_name} has a {prob1:.2%} chance of winning.")
    else:
        print(f"{pokemon2_name} has a {prob2:.2%} chance of winning.")

In [110]:
# remove warning notification
import warnings
warnings.filterwarnings('ignore')

In [118]:
battle('Mega Venusaur', 'Mega Diancie')

Mega Diancie has a 88.54% chance of winning.


In [115]:
battle('Pikachu', 'Mewtwo')

Mewtwo has a 93.39% chance of winning.


In [116]:
battle('Pikachu', 'Charizard')

Charizard has a 85.63% chance of winning.


In [117]:
battle('Agumon', 'Patamon')

Maaf, data tidak ditemukan.
